## File structure
>WORKSPACE_DIR(.)
>>PRJ_NAME.ipynb  
>>PRJ_NAME_get_data.ipynb  
>>PRJ_NAME_train_model.ipynb   
>>DATA_PATH(datasets)  
>>>PRJ_NAME  
>>>>TRAIN_DIR(train)  
>>>>>CLASS_1_DIR  
>>>>>CLASS_n_DIR  

>>>>VALIDATION_DIR(validation)  
>>>>>CLASS_1_DIR  
>>>>>CLASS_n_DIR  

>>>>TEST_DIR(test)
>>>>>CLASS_1_DIR  
>>>>>CLASS_n_DIR  

>>MODEL_PATH(models)  
>>>PRJ_NAME  

>>Image_PATH(images)  
>>>PRJ_NAME  

>>LOG_PATH(logs)  
>>>PRJ_NAME  


In [1]:
import os
import numpy as np
rand_seed=99
# to make this notebook's output stable across runs
np.random.seed(rand_seed)

PRJ_NAME = "face_recognition"

WORKSPACE_DIR = "."
#where to save dataset
DATA_DIR = PRJ_NAME
DATA_PATH = os.path.join( WORKSPACE_DIR,"datasets",DATA_DIR )

#datasets inlcude train data,validation data,test data
TRAIN_DIR = os.path.join( DATA_PATH, 'train' )
VALIDATION_DIR = os.path.join( DATA_PATH, 'validation' )
TEST_DIR = os.path.join( DATA_PATH, 'test' )

#where to save models
MODEL_DIR = PRJ_NAME
MODEL_PATH = os.path.join( WORKSPACE_DIR,"models",MODEL_DIR )

# where to save figures
IMAGE_DIR = PRJ_NAME
IMAGE_PATH = os.path.join( WORKSPACE_DIR,"images",IMAGE_DIR )

#where to save logs
LOG_DIR = PRJ_NAME
LOG_PATH = os.path.join( WORKSPACE_DIR,"logs",LOG_DIR )

In [2]:
#this is a binary classification，
#so every datafile inlcudes two classfiles,jarvis & other
#note: you can use face_recogniton_get_data.ipynb to generate your face dataset
train_jarvis_dir = os.path.join( TRAIN_DIR, 'jarvis' )
train_other_dir = os.path.join( TRAIN_DIR, 'other' )
validation_jarvis_dir = os.path.join( VALIDATION_DIR, 'jarvis' )
validation_other_dir = os.path.join( VALIDATION_DIR, 'other' )
test_jarvis_dir = os.path.join( TEST_DIR, 'jarvis' )
test_other_dir = os.path.join( TEST_DIR, 'other' )
#show the number of datafiles
print('total training jarvis images:', len(os.listdir(train_jarvis_dir)))
print('total training other images:', len(os.listdir(train_other_dir)))
print('total validation jarvis images:', len(os.listdir(validation_jarvis_dir)))
print('total validation other images:', len(os.listdir(validation_other_dir)))
print('total test jarvis images:', len(os.listdir(test_jarvis_dir)))
print('total test other images:', len(os.listdir(test_other_dir)))

total training jarvis images: 389
total training other images: 387
total validation jarvis images: 396
total validation other images: 390
total test jarvis images: 196
total test other images: 196


In [3]:
import cv2
#import face_detect file path from cv2
#you should change this filepath fit your environment
cascade_path = "C:/Users/jarvis/AppData/Local/conda/conda/envs/tensorflow/Library/etc/haarcascades/haarcascade_frontalface_alt2.xml"  
#before import keras,you should make sure TensorFlow, CNTK, or Theano had been installed
import keras
from keras import models

C:\Users\jarvis\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
def CatchVideo(window_name, camera_idx):
    #new a window,the name is window_name
    cv2.namedWindow(window_name)
    
    #open camera
    cap = cv2.VideoCapture(camera_idx)                
    
    #load detect face classfier
    classfier = cv2.CascadeClassifier(cascade_path)
    
    #set green rectangle to show face
    color = (0, 255, 0)
    #load the model whick had been trained
    #note: you can use face_recogniton_train_model.ipynb to train and save model
    model = models.load_model(filepath=MODEL_PATH+"/weights.hdf5")  
    result=0
    while cap.isOpened():
        #read frame
        _, frame = cap.read()
        
        #rgb2gray
        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)                 
        
        #face detect
        faceRects = classfier.detectMultiScale(grey, scaleFactor = 1.2, minNeighbors = 3, minSize = (32, 32))
        if len(faceRects) > 0:                                               
            for faceRect in faceRects:  
                x, y, w, h = faceRect
               
                if (x >= 11) and (y >= 11):#leave distance between edge and face detected
                    img = frame[y - 10: y + h + 10, x - 10: x + w + 10]
                    cv2.rectangle(frame, (x - 10, y - 10), (x + w + 10, y + h + 10), color, 2)

                    img_size= img.shape
#                    print("x ",x,"y ",y,"w ",w,"h ",w,"iw ",img_size[0],"ih ",img_size[1])

                    img = cv2.resize(img,(100,100))                   
                    tmp_img = np.expand_dims(img,axis = 0) / 255
                    result = model.predict(tmp_img)
                    if (result < 0.5):
                        cv2.putText(frame,'jarvis',(x + 20, y + 20),
                                    cv2.FONT_HERSHEY_SIMPLEX,
                                    1,(255,255,255))
                    else:
                        cv2.putText(frame,'other',(x + 20, y + 20),
                                    cv2.FONT_HERSHEY_SIMPLEX,
                                    1,(255,255,255))
                else:
                    pass
                        
        #display frame & press q-key exit
        cv2.imshow(window_name, frame)        
        c = cv2.waitKey(2)
        if c & 0xFF == ord('q'):
            break        
    cap.release()
    cv2.destroyAllWindows() 
    

In [5]:
CatchVideo(PRJ_NAME,0)